# Embarrasingly parallel date operations

each row is processed independently

I want to calculate the number of days to and after the next holiday. As I am new to python I am unsure how to perform such a calculation efficiently

In [1]:
%load_ext Cython

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import datetime as DT

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import time

load dates and holidays

In [4]:
datesFrame = pd.read_csv('myDates.csv')
datesFrame.myDates = pd.to_datetime(datesFrame.myDates)

holidays = pd.read_csv('holidays.csv')
holidays.day = pd.to_datetime(holidays.day)
holidays.type = holidays.type.astype("category")
holidays.name = holidays.name.astype("category")

holidays = holidays[holidays.apply(lambda x: (x.type == 'National holiday'), axis=1)]

In [5]:
def get_nearest_date(dates, pivot):
    nearest = min(dates, key=lambda x: abs(x - pivot))
    difference = abs(nearest - pivot)
    return difference / np.timedelta64(1, 'D')

## approach 1 - too slow

In [6]:
datesFrameSmall= datesFrame[: 10]

In [7]:
time1 = time.time()
datesFrameSmall['daysBeforeHoliday'] = datesFrameSmall.myDates.apply(lambda x: get_nearest_date(holidays.day[holidays.day < x], x))
datesFrameSmall['daysAfterHoliday']  =  datesFrameSmall.myDates.apply(lambda x: get_nearest_date(holidays.day[holidays.day > x], x))
time2 = time.time()
print ('function took %0.3f ms' % ((time2-time1)*1000.0))
datesFrameSmall

function took 114.090 ms


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,myDates,daysBeforeHoliday,daysAfterHoliday
0,2014-09-01,17.0,55.0
1,2014-03-01,54.0,51.0
2,2014-03-01,54.0,51.0
3,2014-01-18,12.0,93.0
4,2014-01-14,8.0,97.0
5,2014-01-23,17.0,88.0
6,2014-12-01,30.0,7.0
7,2014-03-01,54.0,51.0
8,2014-03-01,54.0,51.0
9,2014-04-06,90.0,15.0


In [16]:
# disabled - takes too long
#time1 = time.time()
#datesFrame['daysBeforeHoliday'] = datesFrame.myDates.apply(lambda x: get_nearest_date(holidays.day[holidays.day < x], x))
#datesFrame['daysAfterHoliday']  =  datesFrame.myDates.apply(lambda x: get_nearest_date(holidays.day[holidays.day > x], x))
#time2 = time.time()
#print ('function took %0.3f ms' % ((time2-time1)*1000.0))
#datesFrame

## approach 2 - noReturnValues  -> parallel processing + cython

In [19]:
%%cython
################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import datetime as DT
import skutils

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import time
## strange - > have to re import all the things for cython. 
datesFrame = pd.read_csv('myDates.csv')
datesFrame.myDates = pd.to_datetime(datesFrame.myDates)

holidays = pd.read_csv('holidays.csv')
holidays.day = pd.to_datetime(holidays.day)
holidays.type = holidays.type.astype("category")
holidays.name = holidays.name.astype("category")

holidays = holidays[holidays.apply(lambda x: (x.type == 'National holiday'), axis=1)]

def get_nearest_date(dates, pivot):
    nearest = min(dates, key=lambda x: abs(x - pivot))
    difference = abs(nearest - pivot)
    return difference / np.timedelta64(1, 'D')
################
import multiprocessing
num_cpus = multiprocessing.cpu_count()

time1 = time.time()
n_entries = datesFrame.shape[0]
n_entrie_perfold = round(n_entries / num_cpus)
folds = [datesFrame[start:start+n_entrie_perfold] for start in range(0, n_entries + 1, n_entrie_perfold)]

def process_fold(df_per_fold):
    df_per_fold['daysBeforeHoliday'] = df_per_fold.myDates.apply(lambda x: get_nearest_date(holidays.day[holidays.day < x], x))
    df_per_fold['daysAfterHoliday'] = df_per_fold.myDates.apply(lambda x: get_nearest_date(holidays.day[holidays.day > x], x))
    return df_per_fold
    
pool = multiprocessing.Pool(processes=num_cpus)  
pool.map(process_fold, folds)

time2 = time.time()
print ('function took %0.3f ms' % ((time2-time1)*1000.0))
datesFrame

Seems to parallelize well. Still it is not really "quick". Strange that cython does require everything in a single jupyter cell. 

However, there is no output in the datesframe. What am I doing wrong here?

## approach 3 - parallelApply

In [ ]:
from multiprocessing import Pool, cpu_count

def applyParallel(dfGrouped, func):
    with Pool(cpu_count()) as p:
        ret_list = p.map(func, [group for name, group in dfGrouped])
    return pandas.concat(ret_list)

def apply_row_foo(input_df):
    return input_df.apply((row_foo), axis=1)

n_chunks = 10

grouped = df.groupby(df.index // n_chunks)
applyParallel(grouped, apply_row_foo)

only for grouped data --> how to port to rows?

In [27]:
# from http://stackoverflow.com/questions/26187759/parallelize-apply-after-pandas-groupby/29281494#29281494
from joblib import Parallel, delayed
import multiprocessing

def tmpFunc(df):
    df['c'] = df.a + df.b
    return df

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

df = pd.DataFrame({'a': [6, 2, 2], 'b': [4, 5, 6]},index= ['g1', 'g1', 'g2'])
print ('parallel version: ')
print( applyParallel(df.groupby(df.index), tmpFunc))

print ('regular version: ')
print (df.groupby(df.index).apply(tmpFunc))

parallel version: 
    a  b   c
g1  6  4  10
g1  2  5   7
g2  2  6   8
regular version: 
    a  b   c
g1  6  4  10
g1  2  5   7
g2  2  6   8


here my try to port it. But so far I do not know how to pass both 2 parameters

In [28]:
from joblib import Parallel, delayed
import multiprocessing

def get_nearest_dateParallel(dates, pivot):
    nearest = min(dates, key=lambda x: abs(x - pivot))
    difference = abs(nearest - pivot)
    return difference / np.timedelta64(1, 'D')

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

print ('parallel version: ')
print( applyParallel(datesFrame.groupby(datesFrame.index), get_nearest_dateParallel))

parallel version: 


JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/local/Cellar/python3/3.5.2_1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

.......................................................

## approach 4 - pool

did not have too much time to look into that option. But if approach 3 works that looks much cleaner

In [ ]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
def multiply_columns(data):
    data['length_of_word'] = data['species'].apply(lambda x: len(x))
    return data
    
iris = parallelize_dataframe(iris, multiply_columns)